In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Codemotion 2022: Training a Fraud Detection model

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/gioacchino-martino/cicd-4-ml/blob/main/notebooks/simple_model_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/gioacchino-martino/cicd-4-ml/blob/main/notebooks/simple_model_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/notebook_template.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

In this tutorial, you will simulate a simple model training

### Objective

In this tutorial, you learn how to train a model. 

This tutorial uses the following Google Cloud ML services and resources:

- BigQuery
- Vertex AI

The steps performed include:

- Read the data from BigQuery
- Train a simple scikit-learn model
- Store the artefact on the cloud bucket

### Dataset

The datasets contain transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. It is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group ( http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection.



### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQyuery
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
[BigQuery pricing](https://cloud.google.com/bigquery/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

## Installation

Install the following packages required to execute this notebook. 

{TODO: Suggest using the latest major GA version of each package; i.e., --upgrade}

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade numpy pandas scikit-learn xgboost google-cloud-bigquery google-cloud-aiplatform {USER_FLAG} -q

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: cicd-4ml


In [3]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. It is recommended that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [5]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "europe-west1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [6]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. 

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [7]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS '[your-service-account-key-path]'

### Get your project number

Now that the project ID is set, you get your corresponding project number.

In [8]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

Project Number: 1002168816574


### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

In [9]:
BUCKET_NAME = "cicd-4ml_models"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [10]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [11]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://cicd-4ml_models/...
ServiceException: 409 A Cloud Storage bucket named 'cicd-4ml_models' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [12]:
! gsutil ls -al $BUCKET_URI

        11  2022-10-15T15:20:16Z  gs://cicd-4ml_models/Dockerfile1#1665847216491993  metageneration=1
       170  2022-10-15T15:25:01Z  gs://cicd-4ml_models/cloudbuild.yaml#1665847501679569  metageneration=1
    184601  2022-10-15T16:38:36Z  gs://cicd-4ml_models/model.json#1665851916522348  metageneration=1
    184601  2022-10-17T08:47:09Z  gs://cicd-4ml_models/model.json#1665996429477732  metageneration=1
    184601  2022-10-17T16:04:59Z  gs://cicd-4ml_models/model.json#1666022699311857  metageneration=1
TOTAL: 5 objects, 553984 bytes (541 KiB)


#### Service Account

If you do not want to use your project's Compute Engine service account, set `SERVICE_ACCOUNT` to another service account ID.

In [13]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [14]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 1002168816574-compute@developer.gserviceaccount.com


#### Set service account access

Run the following commands to grant your service account access to the bucket that you created in the previous step. You only need to run this step once per service account.

In [15]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Set project folder

### Import libraries

In [16]:
# Training 
import json
import numpy as np
import pandas as pd
from google.cloud import bigquery
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

### Define constants

In [17]:
BQ_PROJECT = 'bigquery-public-data'
BQ_DATASET = 'ml_datasets'
BQ_TABLE = 'ulb_fraud_detection'

### Initialize BigQuery for Python

Initialize the BigQuery SDK for Python for your project and corresponding bucket.

In [18]:
bq_client = bigquery.Client(project = PROJECT_ID)

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [19]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Train a ML model

The steps will be

*   Read the data from BigQuery
*   Preprocess data 
*   Train a ML Model
*   Store the ML Model on the bucket

### Read the data from BigQuery

In [20]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
raw_df = bq_client.query(query = query).to_dataframe()

In [21]:
raw_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,282.0,-0.356466,0.725418,1.971749,0.831343,0.369681,-0.107776,0.751610,-0.120166,-0.420675,...,0.020804,0.424312,-0.015989,0.466754,-0.809962,0.657334,-0.043150,-0.046401,0.0,0
1,380.0,-1.299837,0.881817,1.452842,-1.293698,-0.025105,-1.170103,0.861610,-0.193934,0.592001,...,-0.272563,-0.360853,0.223911,0.598930,-0.397705,0.637141,0.234872,0.021379,0.0,0
2,820.0,-0.937481,0.401649,1.882689,-0.362001,0.751088,-0.899262,0.880557,-0.181650,-0.211657,...,-0.001757,0.097379,-0.324050,0.436521,0.509674,0.454116,-0.201804,-0.175439,0.0,0
3,1193.0,1.130646,0.625391,0.837987,2.506543,-0.107116,-0.245548,0.099603,-0.041457,-0.867319,...,-0.017154,-0.014311,0.086559,0.393496,0.332062,-0.066378,0.013858,0.025382,0.0,0
4,2371.0,-0.878833,0.133657,2.534047,2.609811,1.510839,2.075778,-0.384729,0.230300,-0.367956,...,-0.296422,-0.255485,-0.583298,-1.677514,0.050524,0.250409,-0.223149,-0.420764,0.0,0


### Preprocess data 

In [22]:
target = 'Class'
predictors = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
              'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
              'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28',
              'Amount']

y = raw_df[target]
X = raw_df[predictors]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8, shuffle=True)

### Train the model

In [25]:
clf = XGBClassifier(
    objective='binary:logistic', 
    eval_metric=['auc','error','logloss'],
    early_stopping_rounds=30,
    )
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])

[0]	validation_0-auc:0.90272	validation_0-error:0.00070	validation_0-logloss:0.43806
[1]	validation_0-auc:0.90274	validation_0-error:0.00067	validation_0-logloss:0.29718
[2]	validation_0-auc:0.90756	validation_0-error:0.00063	validation_0-logloss:0.20845
[3]	validation_0-auc:0.90757	validation_0-error:0.00061	validation_0-logloss:0.14910
[4]	validation_0-auc:0.90757	validation_0-error:0.00065	validation_0-logloss:0.10806
[5]	validation_0-auc:0.90756	validation_0-error:0.00061	validation_0-logloss:0.07909
[6]	validation_0-auc:0.90755	validation_0-error:0.00060	validation_0-logloss:0.05836
[7]	validation_0-auc:0.90755	validation_0-error:0.00060	validation_0-logloss:0.04339
[8]	validation_0-auc:0.90749	validation_0-error:0.00060	validation_0-logloss:0.03251
[9]	validation_0-auc:0.90713	validation_0-error:0.00058	validation_0-logloss:0.02460
[10]	validation_0-auc:0.90712	validation_0-error:0.00058	validation_0-logloss:0.01882
[11]	validation_0-auc:0.92858	validation_0-error:0.00058	validat

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=30, enable_categorical=False,
              eval_metric=['auc', 'error', 'logloss'], gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

### Evaluate the model

In [26]:
y_pred = clf.predict(X_test)
target_names = ['NoFraud', 'Fraud']
print(classification_report(y_test, y_pred, target_names=target_names))


              precision    recall  f1-score   support

     NoFraud       1.00      1.00      1.00     56859
       Fraud       0.93      0.77      0.84       103

    accuracy                           1.00     56962
   macro avg       0.96      0.88      0.92     56962
weighted avg       1.00      1.00      1.00     56962



### Save the model

In [27]:
clf.save_model("model.json")

### Test the model

In [29]:
req = """{"instances" : [
  {"V1":-0.356,"V2":0.725,"V3":1.972,"V4":0.831,"V5":0.37,"V6":-0.108, "V7":0.752,"V8":-0.12,"V9":-0.421,"V10":-0.06,"V11":-0.508,"V12":0.426, "V13":0.414,"V14":-0.698,"V15":-1.465,"V16":-0.119,"V17":-0.145,"V18":-1.332, "V19":-1.547,"V20":-0.134,"V21":0.021,"V22":0.424,"V23":-0.016,"V24":0.467, "V25":-0.81,"V26":0.657,"V27":-0.043,"V28":-0.046,"Amount":0.0}
]}"""
req_dict = json.loads(req)["instances"]
req_df = pd.DataFrame.from_records(req_dict)
clf = XGBClassifier()
clf.load_model("model.json")
print("My prediction: ", clf.predict(req_df))

My prediction:  [0]


### Version the model in Google Cloud storage

In [ ]:
!gsutil cp model.json $BUCKET_URI/model.json

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial

In [ ]:
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI